In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 39.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [7]:
import json
import os
import hashlib
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from collections import defaultdict
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def load_data(path):
    with open(path, 'r', encoding="utf-8") as f:
        data = json.load(f)
    return data

In [5]:
def save_data(data, path):
    with open(path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

In [6]:
path = "/content/drive/MyDrive/Colab Notebooks/Data/full_luat_thuong_mai_crawl.json"
df = load_data(path)
df

{'data': [{'title': 'Điều 1 36/2005/QH11 thương mại',
   'paragraphs': [{'context': 'Phạm vi điều chỉnh 1. Hoạt động thương mại thực hiện trên lãnh thổ nước Cộng hoà xã hội chủ nghĩa Việt Nam. 2. Hoạt động thương mại thực hiện ngoài lãnh thổ nước Cộng hoà xã hội chủ nghĩa Việt Nam trong trường hợp các bên thoả thuận chọn áp dụng Luật này hoặc luật nước ngoài, điều ước quốc tế mà Cộng hoà xã hội chủ nghĩa Việt Nam là thành viên có quy định áp dụng Luật này. 3. Hoạt động không nhằm mục đích sinh lợi của một bên trong giao dịch với thương nhân thực hiện trên lãnh thổ nước Cộng hoà xã hội chủ nghĩa Việt Nam trong trường hợp bên thực hiện hoạt động không nhằm mục đích sinh lợi đó chọn áp dụng Luật này.'}]},
  {'title': 'Điều 2 36/2005/QH11 thương mại',
   'paragraphs': [{'context': 'Đối tượng áp dụng 1. Thương nhân hoạt động thương mại theo quy định tại Điều 1 của Luật này. 2. Tổ chức, cá nhân khác hoạt động có liên quan đến thương mại. 3. Căn cứ vào những nguyên tắc của Luật này, Chính phủ

#**Generate Question**


In [18]:
def question_generation(model, tokenizer, data, batch_size, min_length, max_length):

    print("********** Question Generation Starts! **********")
    ques= []
    questions = []
    temp=[]
    for d in data:
        title = d['title']
        paragraphs = d['paragraphs']
        for paragraph in paragraphs:
            context = paragraph['context']
            temp.append({
                "title": title,
                "context": context,
            })
    for i in range(len(temp)):
        context = temp[i]['context']
        input_ids = tokenizer.encode(context, return_tensors='pt')
        with torch.no_grad():
            beam_outputs = model.generate(
            input_ids=input_ids,
            max_length=128,
            length_penalty=2.0,
            num_beams=5,
            no_repeat_ngram_size=5,
            num_return_sequences=1,
            early_stopping=True
            )
            sampling_outputs = model.generate(
            input_ids=input_ids,
            max_length=64,
            do_sample=True,
            top_p=0.95,
            top_k=10,
            num_return_sequences=1,
            early_stopping=True
           )
        for i in range(len(sampling_outputs)):
          query= tokenizer.decode(sampling_outputs[i], skip_special_tokens=True)
          ques.append(query)
        for i in range(len(beam_outputs)):
          query= tokenizer.decode(beam_outputs[i], skip_special_tokens=True)
          ques.append(query)
        questions.append(ques)
        ques = []

    cqa = {
        "data": []
    }

    title2ctx = {}
    ctx2qas = defaultdict(list)
    titles = [x['title'] for x in temp]

    for title in titles:
        title2ctx[title] = []

    for i in range(len(temp)):
        question = questions[i]
        context = temp[i]['context']
        title = temp[i]['title']

        title2ctx[title].append(context)
        for q in question:
            str2hash = title + context + q
            hash_res = hashlib.md5(str2hash.encode())
            qid = hash_res.hexdigest()

            ctx2qas[context].append({
                "id": qid,
                "question": q +"?",
            })

    for title, ctx in title2ctx.items():
        cqa_index = {
            "title": title,
            "paragraphs": []
        }
        for c in ctx:
            qas = ctx2qas[c]
            paragraph = {
                "context": c,
                "qas": qas
            }
            if paragraph not in cqa_index['paragraphs']:
                cqa_index["paragraphs"].append(paragraph)

        cqa['data'].append(cqa_index)

    print("********** Question Generation Ends! **********")

    return cqa

In [9]:
tokenizer = AutoTokenizer.from_pretrained("doc2query/msmarco-vietnamese-mt5-base-v1")
model = AutoModelForSeq2SeqLM.from_pretrained("doc2query/msmarco-vietnamese-mt5-base-v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [10]:
qg_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [19]:
cqa = question_generation(model, tokenizer, df['data'], batch_size=4, min_length=64, max_length=128)

********** Question Generation Starts! **********
********** Question Generation Ends! **********


In [20]:
cqa

{'data': [{'title': 'Điều 1 36/2005/QH11 thương mại',
   'paragraphs': [{'context': 'Phạm vi điều chỉnh 1. Hoạt động thương mại thực hiện trên lãnh thổ nước Cộng hoà xã hội chủ nghĩa Việt Nam. 2. Hoạt động thương mại thực hiện ngoài lãnh thổ nước Cộng hoà xã hội chủ nghĩa Việt Nam trong trường hợp các bên thoả thuận chọn áp dụng Luật này hoặc luật nước ngoài, điều ước quốc tế mà Cộng hoà xã hội chủ nghĩa Việt Nam là thành viên có quy định áp dụng Luật này. 3. Hoạt động không nhằm mục đích sinh lợi của một bên trong giao dịch với thương nhân thực hiện trên lãnh thổ nước Cộng hoà xã hội chủ nghĩa Việt Nam trong trường hợp bên thực hiện hoạt động không nhằm mục đích sinh lợi đó chọn áp dụng Luật này.',
     'qas': [{'id': '84967eaaaeb87aaa9c281d6e4d844f2f',
       'question': 'định nghĩa về phạm vi điều chỉnh việt nam?'},
      {'id': '454291546d606a4b433b686fb2456b2e',
       'question': 'sửa đổi điều chỉnh tiểu bang?'},
      {'id': '122381023b47e6b15bca1557eb5e8c8a',
       'question':

In [21]:
save_path = "/content/drive/MyDrive/Colab Notebooks/Data/QG_top_k_6.json"
save_data(cqa, save_path)